# Analysis of Displays

## Imports

In [ ]:
import pandas as pd

## Join Information

Here I join the table with the detected objects with the table containing background information about the images.

In [ ]:
detection_df = pd.read_excel('images/2021/tables/detected_objects.xlsx', names=['new_name', 'displays', 'product1', 'product2', 'top_design1', 'bottom_design1', 'top_design2', 'bottom_design2', 'top_design3', 'bottom_design3', 'top_design4', 'bottom_design4', 'shelves', 'nr_stacked_shelves', 'nr_products_without_shelf',
       'max_nr_products_on_shelf'])
info_df = pd.read_excel('images/2021/tables/image_name_information_with_location.xlsx', index_col=0)
df = detection_df.merge(info_df, how='inner', on='new_name')
df.head()

## Display: Design on Top

The data includes images of displays and images of store shelves. This analysis focuses on displays, which is why I delete the other rows.

In [ ]:
display_df = df[df['displays'] > 0]

First, I explore the design of the top part of the displays. One design feature is always present and one of the others is optional.

In [ ]:
retailer_top_design_df = display_df[['retailer', 'new_name', 'top_design1', 'top_design2', 'top_design3', 'top_design4']].copy()
retailer_top_design_df['top_design1'] = display_df['displays'] - display_df['top_design2'] - display_df['top_design3'] - display_df['top_design4']
retailer_top_design_df = retailer_top_design_df.set_index('retailer')
retailer_top_design_df.head()

Here I check whether there are differences between retailers in which headers they set up.

In [ ]:
retailer_top_design_df = retailer_top_design_df.groupby(retailer_top_design_df.index).sum()
retailer_top_design_df.plot.bar();

The different retailers are different to compare because there are huge differences in size. Here I calculate the percentages of different headers for each retailer.

In [ ]:
retailer_top_design_df['total'] = retailer_top_design_df['top_design1'] + retailer_top_design_df['top_design2'] + retailer_top_design_df['top_design3'] + retailer_top_design_df['top_design4']

retailer_top_design_pct_df = pd.DataFrame()
retailer_top_design_pct_df['top_design1'] = retailer_top_design_df['top_design1'] / retailer_top_design_df['total']
retailer_top_design_pct_df['top_design2'] = retailer_top_design_df['top_design2'] / retailer_top_design_df['total']
retailer_top_design_pct_df['top_design3'] = retailer_top_design_df['top_design3'] / retailer_top_design_df['total']
retailer_top_design_pct_df['top_design4'] = retailer_top_design_df['top_design4'] / retailer_top_design_df['total']

retailer_top_design_pct_df.plot.bar(stacked=True);

## Display: Design at Bottom

Now I explore the design of the bottom part of the displays.

In [ ]:
retailer_bottom_design_df = display_df[['retailer', 'new_name', 'bottom_design1', 'bottom_design2']].copy()
retailer_bottom_design_df = retailer_bottom_design_df.set_index('retailer')
retailer_bottom_design_df.head()

Here I check whether there are differences between retailers in which footers they set up.

In [ ]:
retailer_bottom_design_df = retailer_bottom_design_df.groupby(retailer_bottom_design_df.index).sum()
retailer_bottom_design_df.plot.bar();

The different retailers are different to compare because there are huge differences in size. Here I calculate the percentages of different footers for each retailer.

In [ ]:
retailer_bottom_design_df['total'] = retailer_bottom_design_df['bottom_design1'] + retailer_bottom_design_df['bottom_design2']

retailer_bottom_design_pct_df = pd.DataFrame()
retailer_bottom_design_pct_df['bottom_design1'] = retailer_bottom_design_df['bottom_design1'] / retailer_bottom_design_df['total']
retailer_bottom_design_pct_df['bottom_design2'] = retailer_bottom_design_df['bottom_design2'] / retailer_bottom_design_df['total']

retailer_bottom_design_pct_df.plot.bar(stacked=True);

## Display: Product Facings

This part looks at the number of facings of two different products.

In [ ]:
facings_df = display_df[['new_name', 'product1', 'product2', 'retailer']].copy()
facings_df['product1'] = facings_df['product1'] / display_df['displays']
facings_df['product2'] = facings_df['product2'] / display_df['displays']

In [ ]:
facings_df.boxplot(by='retailer', rot=90);

Now I want to get an idea about how the number of facings of the two products relate to each other on a typical shelf.

In [ ]:
retailer_facings_df = facings_df.groupby('retailer').median()
retailer_facings_df.plot.bar();

There are differences in the size of shelves that different retailers use and the number of both the products on the shelves. For a better comparison of the two products, I calculate percentages.

In [ ]:
retailer_facings_df['total'] = retailer_facings_df['product1'] + retailer_facings_df['product2']

retailer_facings_pct_df = pd.DataFrame()
retailer_facings_pct_df['product1'] = retailer_facings_df['product1'] / retailer_facings_df['total']
retailer_facings_pct_df['product2'] = retailer_facings_df['product2'] / retailer_facings_df['total']

retailer_facings_pct_df.plot.bar(stacked=True);

## Display: Correctly Stacked Displays by Retailer

This part checks how many displays contain products that are not supposed to be on the displays for each retailer.

In [ ]:
def correctly_stacked_display_check(nr_shelves, nr_shelves_with_correct_products, nr_of_correct_products_per_shelf):
    correct_check = True
    if nr_shelves > nr_shelves_with_correct_products:
        correct_check = False
    if nr_of_correct_products_per_shelf in (1,2,3):
        correct_check = False
    if (nr_of_correct_products_per_shelf == 4) and (nr_shelves in (6,12)):
        correct_check = False
    return correct_check

In [ ]:
display_df['correctly_stacked'] = display_df[['shelves', 'nr_stacked_shelves', 'max_nr_products_on_shelf']].apply(lambda x: correctly_stacked_display_check(x[0], x[1], x[2]), axis=1)

# These lines are only for other uses of info_df outside this notebook.
detection_df['correctly_stacked'] = detection_df[['shelves', 'nr_stacked_shelves', 'max_nr_products_on_shelf']].apply(lambda x: correctly_stacked_display_check(x[0], x[1], x[2]), axis=1)
detection_df.to_excel('images/2021/tables/detected_objects.xlsx')


In [ ]:
correctly_stacked_df = display_df[['new_name', 'retailer', 'correctly_stacked']].copy()
correct_df = correctly_stacked_df[correctly_stacked_df['correctly_stacked'] == True].groupby('retailer')['correctly_stacked'].count().to_frame()
incorrect_df = correctly_stacked_df[correctly_stacked_df['correctly_stacked'] == False].groupby('retailer')['correctly_stacked'].count().to_frame()
incorrect_df.columns = ['incorrectly_stacked']
retailer_correctly_stacked_df = correct_df.join(incorrect_df)
retailer_correctly_stacked_df['total'] = retailer_correctly_stacked_df['correctly_stacked'] + retailer_correctly_stacked_df['incorrectly_stacked']

retailer_correctly_stacked_pct_df = pd.DataFrame()
retailer_correctly_stacked_pct_df['correctly_stacked'] = retailer_correctly_stacked_df['correctly_stacked'] / retailer_correctly_stacked_df['total']
retailer_correctly_stacked_pct_df['incorrectly_stacked'] = retailer_correctly_stacked_df['incorrectly_stacked'] / retailer_correctly_stacked_df['total']

retailer_correctly_stacked_pct_df.plot.bar(stacked=True);

## Display: Correctly Stacked Displays by Location

This part checks how many displays contain products that are not supposed to be on the displays for each region.

In [ ]:
correctly_stacked_df = display_df[['new_name', 'province', 'correctly_stacked']].copy()
correct_df = correctly_stacked_df[correctly_stacked_df['correctly_stacked'] == True].groupby('province')['correctly_stacked'].count().to_frame()
incorrect_df = correctly_stacked_df[correctly_stacked_df['correctly_stacked'] == False].groupby('province')['correctly_stacked'].count().to_frame()
incorrect_df.columns = ['incorrectly_stacked']
province_correctly_stacked_df = correct_df.join(incorrect_df)
province_correctly_stacked_df['total'] = province_correctly_stacked_df['correctly_stacked'] + province_correctly_stacked_df['incorrectly_stacked']

province_correctly_stacked_pct_df = pd.DataFrame()
province_correctly_stacked_pct_df['correctly_stacked'] = province_correctly_stacked_df['correctly_stacked'] / province_correctly_stacked_df['total']
province_correctly_stacked_pct_df['incorrectly_stacked'] = province_correctly_stacked_df['incorrectly_stacked'] / province_correctly_stacked_df['total']

province_correctly_stacked_pct_df.plot.bar(stacked=True);